In [1]:
import numpy as np
import pandas as pd 
import os
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.fft import fft
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
from pyod.models.knn import KNN
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_absolute_error, mean_squared_error
from datetime import datetime 

import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_selection import VarianceThreshold, RFE
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LassoCV

In [2]:
def convert_sliding_window_data(data, target_column, window_size):
    df_sliding_window = data.copy()
    new_elements = df_sliding_window.columns.tolist()
    new_elements.remove("时间")
    
    # 每个变量都进行shift
    shift_cols = []
    shift_col_names = []
    for i in range(len(new_elements)):
        for window in window_size:
            shift_cols.append(df_sliding_window[new_elements[i]].shift(window))
            shift_col_names.append(f'{new_elements[i]}_{window}')
    
    # 按照window计算统计特征
    stats_cols = []
    stats_col_names = []
    for window in window_size[:-2]:
        f_min = lambda x: x.rolling(window=window).min()
        f_max = lambda x: x.rolling(window=window).max()
        f_mean = lambda x: x.rolling(window=window).mean()
        f_std = lambda x: x.rolling(window=window).std()
        f_median = lambda x: x.rolling(window=window).median()
        f_skew = lambda x:x.rolling(window=window).skew()
        function_list = [f_min, f_max, f_mean, f_std, f_median, f_skew]
        function_name = ['min', 'max', 'mean', 'std', 'median', 'skew']
        for k in range(len(function_list)):
            stats_cols.append(df_sliding_window[new_elements].apply(function_list[k]).shift(1))
            stats_col_names.extend(pd.Series(df_sliding_window[new_elements].columns)
                                   .apply(lambda x:x+str(f'_rolling_{window}_{function_name[k]}')).tolist())
            
    x_shift = pd.concat(shift_cols, axis=1)
    x_shift.columns = shift_col_names

    x_stats = pd.concat(stats_cols, axis=1)
    x_stats.columns = stats_col_names
    
    df_sliding_window = pd.concat([df_sliding_window, x_shift, x_stats], axis=1)
            
    # 提取时间性的特征
    df_sliding_window['Day'] = df_sliding_window['时间'].dt.day
    df_sliding_window['DayOfWeek'] = df_sliding_window['时间'].dt.dayofweek
    df_sliding_window['Hour'] = df_sliding_window['时间'].dt.hour 
    df_sliding_window['minute'] = df_sliding_window['时间'].dt.minute 
    
    # 缺失值剔除+原变量剔除(因变量除外)
    new_elements.remove(target_column[0])
    df_sliding_window = df_sliding_window.dropna(axis=0, how='any').drop(new_elements, axis=1)
    return df_sliding_window.reset_index(drop=True)

In [3]:
def make_window_predict_data(data, window_size):
    max_window = max(window_size)
    df_tobe_predicted = data.tail(max_window).copy()
    new_elements = df_tobe_predicted.columns.tolist()
    new_elements.remove("时间")
    
    shift_cols = []
    shift_col_names = []
    for i in range(len(new_elements)):
        for window in window_size[1:]:
            shift_cols.append(df_tobe_predicted[new_elements[i]].shift(window))
            shift_col_names.append(f'{new_elements[i]}_{window+1}')
    
    stats_cols = []
    stats_col_names = []
    for window in window_size[:-2]:
        f_min = lambda x: x.rolling(window=window).min()
        f_max = lambda x: x.rolling(window=window).max()
        f_mean = lambda x: x.rolling(window=window).mean()
        f_std = lambda x: x.rolling(window=window).std()
        f_median = lambda x: x.rolling(window=window).median()
        f_skew = lambda x:x.rolling(window=window).skew()
        function_list = [f_min, f_max, f_mean, f_std, f_median, f_skew]
        function_name = ['min', 'max', 'mean', 'std', 'median', 'skew']
        for k in range(len(function_list)):
            stats_cols.append(df_tobe_predicted[new_elements].apply(function_list[k]))
            stats_col_names.extend(pd.Series(df_tobe_predicted[new_elements].columns)
                                   .apply(lambda x:x+str(f'_rolling_{window}_{function_name[k]}')).tolist())  
            
    x_shift = pd.concat(shift_cols, axis=1)
    x_shift.columns = shift_col_names

    x_stats = pd.concat(stats_cols, axis=1)
    x_stats.columns = stats_col_names
    df_tobe_predicted = pd.concat([df_tobe_predicted, x_shift, x_stats], axis=1).dropna(axis=0)
    
    df_tobe_predicted['Day'] = df_tobe_predicted['时间'].dt.day
    df_tobe_predicted['DayOfWeek'] = df_tobe_predicted['时间'].dt.dayofweek
    df_tobe_predicted['Hour'] = df_tobe_predicted['时间'].dt.hour 
    df_tobe_predicted['minute'] = df_tobe_predicted['时间'].dt.minute 
    
    df_tobe_predicted.rename({new_elements[i]: new_elements[i]+str(f'_1') for i in range(len(new_elements))}, axis=1, inplace=True)
    return df_tobe_predicted

In [32]:
def fit_sliding_window_model(data:pd.DataFrame, internal_factors:list, external_factors:list, target_column:list):
    # 数据类型更改和校验、数据拼接
    data['时间'] = pd.to_datetime(data['时间'])
    data = data.infer_objects()
    data.columns =[col.strip() for col in data.columns]
    internal_factors = list(map(str.strip, internal_factors))
    external_factors = list(map(str.strip, external_factors))
    target_column = list(map(str.strip, target_column))
    print(target_column)
    
    df = data[list(set(['时间'] + internal_factors + external_factors + target_column))]
    # 导入数据的维度
    shape1 = str(df.shape)
    
    # 根据启机条件筛选数据、缺失值填充
    df = df.query("`10MKA10CE301XQ01` > 50")
    df = df.fillna(df.mean()).reset_index(drop=True)
    # 筛选后数据的维度
    shape2 = str(df.shape)
    
    df_train = df.query("时间 < '2021-01-28 00:00:00'").reset_index(drop=True)
    df_val = df.query("时间 >= '2021-01-28 00:00:00' & 时间 < '2021-01-30 00:00:00'").reset_index(drop=True)
    df_test = df.query("时间 >= '2021-01-30 00:00:00'").reset_index(drop=True)
    # 训练集数据
    shape3 = str(df_train.shape)
    # 验证集数据
    shape4 = str(df_val.shape)
    # 测试集数据
    shape5 = str(df_test.shape)

    # 特征提取
    window_size=[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
    df_train_window = convert_sliding_window_data(df_train, target_column, window_size)
    df_val_window = convert_sliding_window_data(df_val, target_column, window_size)
    df_test_window = convert_sliding_window_data(df_test, target_column, window_size)
    train_window_shape = str(df_train_window.shape)
    val_window_shape = str(df_val_window.shape)
    test_window_shape = str(df_test_window.shape)
    
    # 制作未来半个小时待预测的数据集
    val_predict = make_window_predict_data(df_val, window_size)
    test_predict = make_window_predict_data(df_test, window_size)
    
    # 定义自变量和因变量
    train_cols = df_train_window.columns.tolist()
    train_cols.remove('时间')
    train_cols.remove(target_column[0])

    x_train, y_train = df_train_window[train_cols], df_train_window[target_column]
    x_val, y_val = df_val_window[train_cols], df_val_window[target_column]
    x_test, y_test = df_test_window[train_cols], df_test_window[target_column]
    
    # 特征选择
    reg = RandomForestRegressor(n_jobs=-1) 
    reg.fit(x_train.values, y_train.values) 
    feature_model = SelectFromModel(reg, prefit=True, max_features=int(x_train.shape[1]*0.8))
    x_train_new = feature_model.transform(x_train)
    model_selected_columns = x_train.columns[feature_model.get_support()]
    shape6 = str(x_train_new.shape)
    
    # 随机森林
    pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=-999)),
    ('model', RandomForestRegressor())])
    param_grid = {'model__n_estimators':[*range(10, 250, 10)],
                  'model__max_depth':[*range(5, 80, 5)]}

    # 网格搜索
    grid = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=3, n_jobs=-1) 
    grid.fit(x_train_new, y_train.values)
    
    # 保存模型
    joblib.dump(grid, f'D:/Jupyterfiles/slidingwindow_model/ybl_{target_column[0]}.pkl')
    
    # 在验证集上的结果(mape,r2,mae,mse)、真实值和预测值的最大差异
    pred_val_value = grid.predict(x_val[model_selected_columns].values)
    val_mape = mean_absolute_percentage_error(y_val.values, pred_val_value)
    val_r2 = r2_score(y_val.values, pred_val_value)
    val_mae = mean_absolute_error(y_val.values, pred_val_value)
    val_mse = mean_squared_error(y_val.values, pred_val_value)
    val_max_diff_value = max(abs(y_val.values.ravel() - pred_val_value))

    # 在测试集上的结果(mape,r2,mae,mse)、真实值和预测值的最大差异
    pred_test_value = grid.predict(x_test[model_selected_columns].values)
    test_mape = mean_absolute_percentage_error(y_test.values, pred_test_value)
    test_r2 = r2_score(y_test.values, pred_test_value)
    test_mae = mean_absolute_error(y_test.values, pred_test_value)
    test_mse = mean_squared_error(y_test.values, pred_test_value)
    test_max_diff_value = max(abs(y_test.values.ravel() - pred_test_value))

    # 未来半个小时的数据预测
    val_pred_value = grid.predict(val_predict[model_selected_columns].values)[0]
    test_pred_value = grid.predict(test_predict[model_selected_columns].values)[0]

    res_featured = pd.DataFrame({'因变量': target_column,
                        '原始数据':shape1, '筛选后':shape2, '训练集':shape3, 
                        '验证集':shape4, '测试集':shape5, '训练集特征':train_window_shape,
                        '验证集特征':val_window_shape, '测试集特征':test_window_shape, '特征选择后训练集':shape6, 
                                 
                         'val_mape':val_mape, 'val_r2':val_r2, 'val_mae':val_mae, 'val_mse':val_mse, 
                         'val_max_diff_value':val_max_diff_value, 'val_pred_value':val_pred_value,
                                 
                        'test_mape':test_mape, 'test_r2':test_r2, 'test_mae':test_mae, 'test_mse':test_mse,
                        'test_max_diff_value':test_max_diff_value, 'test_pred_value':test_pred_value}, index=[1])
    return res_featured

In [5]:
root = 'D:/Jupyterfiles/ThermalData/2021-01/'
file_paths = os.listdir(root)

data = pd.read_excel(os.path.join(root, file_paths[0]))
for i in range(1, len(file_paths)):
    single_file_data = pd.read_excel(os.path.join(root, file_paths[i]))
    data = pd.merge(data, single_file_data, how='left', on=['时间'])

In [33]:
internal_factors = ['10MKA10CE301XQ01', '10MKA10CE301XQ02', '10MKA10CE101XQ01', '10MKA10CE101XQ02', '10MKA10CE101XQ03',
                  '10MKF30CT302', '10MKG10CT301', '10MKG10CT303', '10MKG10CT305', '10MKG10CT307']

external_factors = ['10MKA01CT301','10MKA01CT303','10MKA01CT305','10MKA01CT307','10MKA01CT309','10MKA01CT311',
                    '10MKA01CT313','10MKA01CT315','10MKA01CT317','10MKA01CT319','10MKA01CT321','10MKA01CT323',
                    '10MKA01CT325','10MKA01CT327','10MKA01CT329','10MKA01CT331','10MKA01CT333','10MKA01CT335',
                    '10MKA01CT337','10MKA01CT339','10MKA01CT341','10MKA01CT343','10MKA01CT345','10MKA01CT347',
                    '10MKA01CT349','10MKA01CT351','10MKA01CT353','10MKA01CT355','10MKA01CT357','10MKA01CT359',
                    '10MKA01CT361','10MKA01CT363','10MKA01CT365','10MKA01CT367','10MKA01CT369','10MKA01CT371','10MKA01CT373',
                    '10MKA01CT375','10MKA01CT377','10MKA01CT379','10MKA01CT381','10MKA01CT383','10MKA01CT601','10MKA01CT602',
                    '10MKA01CT603','10MKA01CT604','10MKA01CT605','10MKA01CT606','10MKA01CT607','10MKA01CT608','10MKA01CT609',
                    '10MKA01CT610','10MKA01CT611','10MKA01CT612','10MKA01CT613','10MKA01CT614','10MKA01CT615','10MKA01CT616',
                    '10MKA01CT617','10MKA01CT618']

In [35]:
res_all = pd.DataFrame(columns=['因变量', '原始数据', '筛选后', '训练集', '验证集', '测试集', 
                                '训练集特征', '验证集特征', '测试集特征', '特征选择后训练集', 
                                'val_mape', 'val_r2', 'val_mae', 'val_mse', 'val_max_diff_value', 'val_pred_value',
                                'test_mape', 'test_r2', 'test_mae', 'test_mse', 'test_max_diff_value', 'test_pred_value'])

In [45]:
time1 = datetime.now()
for i in tqdm(range(21, len(external_factors))): 
    print('当前因变量为：', external_factors[i])
    res = fit_sliding_window_model(data=data, internal_factors=internal_factors, external_factors=external_factors, 
                                   target_column=[external_factors[i]])
    res_all = pd.concat([res_all, res], axis=0, ignore_index=True)
    print('*'*50)
    
time2 = datetime.now()
print('一共耗时：', time2-time1)

  0%|                                                                                           | 0/39 [00:00<?, ?it/s]

当前因变量为： 10MKA01CT343
['10MKA01CT343']


  3%|██                                                                              | 1/39 [08:07<5:08:34, 487.21s/it]

**************************************************
当前因变量为： 10MKA01CT345
['10MKA01CT345']


  5%|████                                                                            | 2/39 [17:16<5:22:47, 523.46s/it]

...

**************************************************
当前因变量为： 10MKA01CT617
['10MKA01CT617']


 97%|████████████████████████████████████████████████████████████████████████████▉  | 38/39 [7:13:46<15:26, 926.04s/it]

**************************************************
当前因变量为： 10MKA01CT618
['10MKA01CT618']


100%|███████████████████████████████████████████████████████████████████████████████| 39/39 [7:22:25<00:00, 680.66s/it]

**************************************************
一共耗时： 7:22:25.692022


In [41]:
pd.set_option('display.max_columns', None)

In [47]:
# res_all.to_csv('sliding_window_res.csv')

In [48]:
res_all

,因变量,原始数据,筛选后,训练集,验证集,测试集,训练集特征,验证集特征,测试集特征,特征选择后训练集,val_mape,val_r2,val_mae,val_mse,val_max_diff_value,val_pred_value,test_mape,test_r2,test_mae,test_mse,test_max_diff_value,test_pred_value
0,10MKA01CT301,"(1489, 71)","(1489, 71)","(1296, 71)","(96, 71)","(97, 71)","(1286, 4066)","(86, 4066)","(87, 4066)","(1286, 123)",0.012662,0.867149,0.678577,0.974362,3.498969,50.020473,0.015182,0.790903,0.814941,1.445541,3.964884,50.193784
1,10MKA01CT303,"(1489, 71)","(1489, 71)","(1296, 71)","(96, 71)","(97, 71)","(1286, 4066)","(86, 4066)","(87, 4066)","(1286, 115)",0.013089,0.869403,0.709685,0.989995,3.396297,50.741493,0.015927,0.792195,0.867159,1.485402,3.890686,50.735213
2,10MKA01CT305,"(1489, 71)","(1489, 71)","(1296, 71)","(96, 71)","(97, 71)","(1286, 4066)","(86, 4066)","(87, 4066)","(1286, 110)",0.013759,0.881300,0.761102,1.155132,3.686234,51.289426,0.016656,0.813504,0.923092,1.693025,4.306253,51.285463
3,10MKA01CT307,"(1489, 71)","(1489, 71)","(1296, 71)","(96, 71)","(97, 71)","(1286, 4066)","(86, 4066)","(87, 4066)","(1286, 114)",0.012366,0.868841,0.661667,0.829022,2.883504,50.072097,0.014816,0.783366,0.791392,1.272705,3.634214,50.198192
4,10MKA01CT309,"(1489, 71)","(1489, 71)","(1296, 71)","(96, 71)","(97, 71)","(1286, 4066)","(86, 4066)","(87, 4066)","(1286, 112)",0.012255,0.869540,0.659827,0.856723,3.484644,50.528831,0.014764,0.776160,0.797383,1.370183,3.821967,50.500990
5,10MKA01CT311,"(1489, 71)","(1489, 71)","(1296, 71)","(96, 71)","(97, 71)","(1286, 4066)","(86, 4066)","(87, 4066)","(1286, 112)",0.012729,0.871635,0.686680,0.971745,3.538271,50.499923,0.014713,0.806388,0.796190,1.374642,3.923888,50.459960
6,10MKA01CT313,"(1489, 71)","(1489, 71)","(1296, 71)","(96, 71)","(97, 71)","(1286, 4066)","(86, 4066)","(87, 4066)","(1286, 105)",0.011819,0.872505,0.639944,0.816844,3.059130,50.513702,0.016116,0.743434,0.875349,1.481324,4.088207,50.589513
7,10MKA01CT315,"(1489, 71)","(1489, 71)","(1296, 71)","(96, 71)","(97, 71)","(1286, 4066)","(86, 4066)","(87, 4066)","(1286, 118)",0.012659,0.873122,0.680773,0.905850,3.482657,50.414962,0.015310,0.786498,0.825091,1.439045,3.754972,50.494254
8,10MKA01CT317,"(1489, 71)","(1489, 71)","(1296, 71)","(96, 71)","(97, 71)","(1286, 4066)","(86, 4066)","(87, 4066)","(1286, 103)",0.012810,0.874963,0.694601,0.965138,3.306254,50.747419,0.016371,0.781064,0.890841,1.592385,3.975915,50.819113
9,10MKA01CT319,"(1489, 71)","(1489, 71)","(1296, 71)","(96, 71)","(97, 71)","(1286, 4066)","(86, 4066)","(87, 4066)","(1286, 105)",0.012887,0.875789,0.699854,1.005040,3.244045,50.682395,0.015431,0.810127,0.839543,1.429956,3.931084,50.723648
